<a href="https://colab.research.google.com/github/valkova-k/cactus-repo/blob/br01/instruments_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


**2. Data Preparation**

In [ ]:
# Converting images to tensors
transform = transforms.ToTensor()

# Connecting the drive folder
from google.colab import drive
drive.mount('/content/drive')

# Loading the training data
train_set = datasets.ImageFolder(root='/content/drive/MyDrive/data_instruments/dataset/train',
                                 transform=transform)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=256, shuffle=True)

# Download and load the test data
test_set = datasets.ImageFolder(root='/content/drive/MyDrive/data_instruments/dataset/test',
                                 transform=transform)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=256, shuffle=False)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train_set

Dataset ImageFolder
    Number of datapoints: 752
    Root location: /content/drive/MyDrive/data_instruments/dataset/train
    StandardTransform
Transform: ToTensor()

**3. Neural Network Model**

In [ ]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3,
                               padding=pad)
        # added padding so that the images don't shrink; when (non-rigorously)
        # tested the models tended to perform better, accuracy-wise
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3,
                               padding=pad)
        self.flat = nn.Flatten()
        self.fc1 = nn.Linear(in_features=28*28*64, out_features=128)
        # 28*28, because the padding prevented shrinkage
        self.drop = nn.Dropout(0.25)
        self.fc2 = nn.Linear(in_features=128, out_features=10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.flat(x)
        x = F.relu(self.fc1(x))
        x = self.drop(x)
        x = self.fc2(x)
        return x

model = SimpleCNN().to(device)
model

SimpleCNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (flat): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=50176, out_features=128, bias=True)
  (drop): Dropout(p=0.25, inplace=False)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)

**4. Loss & Optimizer**

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.02)

**5. Training loop**

In [ ]:
for epoch in range(5):  # Train for 5 epochs
    running_loss = 0.0
    for images, labels in train_loader:
        # Move images and labels to the device
        images = images.to(device)
        labels = labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch [{epoch+1}/5], Loss: {running_loss / len(train_loader):.4f}')

Epoch [1/5], Loss: 2.1843
Epoch [2/5], Loss: 1.8664
Epoch [3/5], Loss: 1.7603
Epoch [4/5], Loss: 1.7368
Epoch [5/5], Loss: 1.7163


**6. Evaluation on the test set**

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {100 * correct / total:.2f}%')

Accuracy: 26.40%
